In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, TensorDataset, random_split
import matplotlib.pyplot as plt


In [76]:

# Define the MLP model
# Define the MLP model
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size=128, num_discrete_actions=20):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, int(2*hidden_size))
        self.fc3 = nn.Linear(int(2*hidden_size), hidden_size)
        self.relu = nn.ReLU()
        
        # Output layers
        self.fc_action = nn.Linear(hidden_size, num_discrete_actions)  # Discrete action classification
        self.fc_heading = nn.Linear(hidden_size, 1)  # Continuous heading direction
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.relu(x)
        action_logits = self.fc_action(x)  # No activation (will apply softmax in loss function)
        heading = self.fc_heading(x)  # No activation (direct regression for heading)
        return action_logits, heading
    


class MLP_continuous(nn.Module):
    def __init__(self, input_size=4, output_size=2):
        super(MLP_continuous, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 256)
        self.fc3 = nn.Linear(256, 256)
         
        # Output layers
        self.fc_action = nn.Linear(256, 20)  # Discrete action classification
        self.fc_heading = nn.Linear(256, 1)  # Continuous heading direction
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.relu(x)
        action = self.relu(self.fc_action(x))
        heading = self.fc_heading(x)
        return action, heading



def load_dataset(state_data, action_data, heading_data, batch_size=64, train_split=0.8):
    states = torch.tensor(state_data, dtype=torch.float32)
    actions = torch.tensor(action_data, dtype=torch.long)  # Discrete actions as integer labels
    headings = torch.tensor(heading_data, dtype=torch.float32)
    dataset = TensorDataset(states, actions, headings)
    
    train_size = int(train_split * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    return train_loader, test_loader


In [77]:
state_data = np.load("all_pos.npy")
heading_data = np.load('all_alpha.npy')
action_data = np.load('all_frq.npy')



In [78]:
train_loader, test_loader = load_dataset(state_data, action_data, heading_data)

In [79]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = MLP(input_size=4, num_discrete_actions=20).to(device)
model = MLP_continuous(input_size=4, output_size=2).to(device)
criterion_action = nn.CrossEntropyLoss()
# criterion_action = nn.MSELoss()
criterion_heading = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)




# Training loop
def train_model(model, train_loader, test_loader, criterion_action, criterion_heading, optimizer, num_epochs=5):
    model.train()
    train_loss_history = []
    test_loss_history = []
    
    for epoch in range(num_epochs):
        total_train_loss = 0
        weights = 60
        for states, actions, headings in train_loader:
            states = states.to(device)
            actions = actions.to(device).float()
            headings = headings.to(device)
            
            optimizer.zero_grad()
            action_logits, heading_pred = model(states)
            
            loss_action = criterion_action(action_logits.squeeze(), actions.long())
            loss_heading = criterion_heading(weights*heading_pred.squeeze(), weights*headings)
            loss = loss_action +  loss_heading
            
            loss.backward()
            optimizer.step()
            
            total_train_loss += loss.item()
        avg_train_loss = total_train_loss / len(train_loader)
        train_loss_history.append(avg_train_loss)
        
        # Evaluate on test data
        model.eval()
        total_test_loss = 0
        avg_test_loss = 0
        with torch.no_grad():
            for states, actions, headings in test_loader:
                states = states.to(device)
                actions = actions.to(device)
                headings = headings.to(device)
                action_logits, heading_pred = model(states)
                
                loss_action = criterion_action(action_logits, actions.long())
                loss_heading = criterion_heading(heading_pred.squeeze(), headings)
                loss = loss_action + loss_heading
                total_test_loss += loss.item()
        avg_test_loss = total_test_loss / len(test_loader)
        test_loss_history.append(avg_test_loss)
        
        print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Test Loss: {avg_test_loss:.4f}")
        
    # Plot loss function
    plt.plot(range(1, num_epochs + 1), train_loss_history, marker='o', label='Train Loss')
    plt.plot(range(1, num_epochs + 1), test_loss_history, marker='s', label='Test Loss')
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title("Training and Test Loss Over Epochs")
    plt.legend()
    plt.grid()
    plt.show()

# Run training
train_model(model, train_loader, test_loader, criterion_action, criterion_heading, optimizer, num_epochs=50)

# Run training


Epoch [1/50], Train Loss: 8979.1897, Test Loss: 4.6933
Epoch [2/50], Train Loss: 7747.7999, Test Loss: 4.4855
Epoch [3/50], Train Loss: 7483.0420, Test Loss: 4.3625
Epoch [4/50], Train Loss: 7336.5669, Test Loss: 4.2472
Epoch [5/50], Train Loss: 7236.6977, Test Loss: 4.2101
Epoch [6/50], Train Loss: 7158.6290, Test Loss: 4.2568
Epoch [7/50], Train Loss: 7098.8899, Test Loss: 4.1529
Epoch [8/50], Train Loss: 7035.1731, Test Loss: 4.1683
Epoch [9/50], Train Loss: 6988.4838, Test Loss: 4.1140
Epoch [10/50], Train Loss: 6941.0660, Test Loss: 4.0986
Epoch [11/50], Train Loss: 6897.2750, Test Loss: 4.0827
Epoch [12/50], Train Loss: 6859.1619, Test Loss: 4.0755
Epoch [13/50], Train Loss: 6821.4652, Test Loss: 4.0985
Epoch [14/50], Train Loss: 6787.6444, Test Loss: 4.0711
Epoch [15/50], Train Loss: 6762.5361, Test Loss: 4.0834
Epoch [16/50], Train Loss: 6728.8118, Test Loss: 4.0144
Epoch [17/50], Train Loss: 6701.0492, Test Loss: 4.0217
Epoch [18/50], Train Loss: 6667.7885, Test Loss: 4.0181
E

KeyboardInterrupt: 

In [115]:
# torch.save(model.state_dict(), "mlp_model.pth")
torch.save(model.state_dict(), "mlp_model_continuous.pth")
print("Model saved successfully.")

Model saved successfully.


In [81]:
def evaluate_model(model, test_loader, criterion):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for states, actions in test_loader:
            states = states.to(device)
            actions = actions.to(device)
            outputs = model(states)
            loss = criterion(outputs, actions)
            total_loss += loss.item()
    print(f"Test Loss: {total_loss/len(test_loader):.4f}")

def infer_action(model, state):
    model.eval()
    state = torch.tensor(state, dtype=torch.float32).to(device).unsqueeze(0)  # Add batch dimension
    with torch.no_grad():
        action_logits, heading = model(state)
        action_probabilities = torch.softmax(action_logits, dim=1)
        action = torch.argmax(action_probabilities, dim=1).item()  # Get the discrete action
        heading = heading.item()  # Get the continuous heading value
    return action, heading

In [86]:
idx = np.random.randint(len(test_loader.dataset))
print('idx:', idx)
sample_state = test_loader.dataset[idx][0].to(device)
sample_action = test_loader.dataset[idx][1].to(device)
sample_heading = test_loader.dataset[idx][2].to(device)
action_out, heading_out = infer_action(model, sample_state)
print("Predicted Action:", action_out, "True Action:", sample_action.item())
print("Predicted Heading:", heading_out, "True Heading:", sample_heading.item())


idx: 182776
Predicted Action: 8 True Action: 8
Predicted Heading: 1.591921329498291 True Heading: -0.39269909262657166


/tmp/ipykernel_3977939/834229402.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  state = torch.tensor(state, dtype=torch.float32).to(device).unsqueeze(0)  # Add batch dimension


In [114]:
idx = np.random.randint(len(train_loader.dataset))
print('idx:', idx)
sample_state = train_loader.dataset[idx][0].to(device)
sample_action = train_loader.dataset[idx][1].to(device)
sample_heading = train_loader.dataset[idx][2].to(device)
action_out, heading_out = infer_action(model, sample_state)
print("Predicted Action:", action_out, "True Action:", sample_action.item())
print("Predicted Heading:", heading_out*180/np.pi, "True Heading:", sample_heading.item()/np.pi*180)


idx: 714255
Predicted Action: 4 True Action: 2
Predicted Heading: 63.3470129290118 True Heading: 180.00000500895632


/tmp/ipykernel_3977939/834229402.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  state = torch.tensor(state, dtype=torch.float32).to(device).unsqueeze(0)  # Add batch dimension
